In [1]:
#  SPARK START!
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("dATA Quality data index") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 18:42:43 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/11/20 18:42:43 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/11/20 18:42:43 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/11/20 18:42:43 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
import math
from pyspark.sql.functions import col, when, count,isnan
import hashlib
import time

hashlib.sha1().update(str(time.time()).encode("utf-8"))
print(hashlib.sha1().hexdigest())
print(hashlib.sha1().hexdigest()[:10])

da39a3ee5e6b4b0d3255bfef95601890afd80709
da39a3ee5e


In [71]:

class DataQualityIndexOf:
    
    def __init__(self, path_to_csv=None):
        self.table_df = spark.read.csv(path=path_to_csv,header=True)
        
        hashlib.sha1().update(str(time.time()).encode("utf-8"))
        self.table_name = "table_"+hashlib.sha1().hexdigest()[:5]
        
        self.table_df.createOrReplaceTempView(self.table_name)
        self.uniq = None
        self.complete = None
        self.complete_c = []
        self.unique_c = []
        
    def load_from_csv(self, path_to_csv):
        self.table_df = spark.read.csv(path=path_to_csv,header=True)
        
        hashlib.sha1().update(str(time.time()).encode("utf-8"))
        self.table_name = "table_"+hashlib.sha1().hexdigest()[:5]
        
        self.table_df.createOrReplaceTempView(self.table_name)
        
    def uniqueness(self):
        total_rows = self.table_df.count()
        
        max_uniq= 0
        
        #constant number of columns so without losing much efficieny we can execute the below
        #we can utilize the .map fnction but then we would have to convert the dataframe to a rdd first anyways which takes time
        self.unique_c = []
        for col in self.table_df.columns:
            
            uniq_i = self.table_df.select(col).distinct().count() / total_rows
            # columns with the highest uniqueness
            self.unique_c.append(uniq_i)
            max_uniq = max(uniq_i,max_uniq) 
        #floor
        
        if max_uniq==1:
            self.uniq=1
            return max_uniq
        self.uniq=0
        return 0
    
    def completeness(self):
        total_rows = self.table_df.count()
        
        total_complete = 0
        complete_c = 0
        
        self.complete_c = []
        for c in self.table_df.columns:
            
            df2 = parking_violation.table_df.filter( col(c) != "Null").filter(col(c) !='None').filter(col(c) != '' ).filter(col(c) != '-' ).filter(col(c).isNotNull() ) 
            
            complete_c = df2.count() / total_rows
            self.complete_c.append(complete_c)
            
            total_complete += complete_c
        
        
        self.complete= total_complete / len(self.table_df.columns)
        return self.complete
    def getDQA(self):
        uniq = self.uniqueness()
        comp = self.completeness()
        return uniq+comp

In [72]:
parking_violation = DataQualityIndexOf('/shared/CS-GY-6513/parking-violations/parking-violations-header.csv')

In [74]:
parking_violation.getDQA()

1.8949280660257894

In [75]:
print(parking_violation.complete)

0.8949280660257893


In [76]:
print(parking_violation.uniq)

1
